In [5]:
#!/usr/bin/env python3

import pandas as pd
import errno    
import os
import numpy as np
import math

'''
Author: Enrico Ceccolini
    assing to each timepoint the number of active cores and active jobs
    by looking each job run_start_time and end_time.
    
'''

datadir = "/datasets/eurora_data/db_local/"
#datadir = "/datasets/eurora_data/db/"

infile_jobs_to_nodes = datadir + "job_nodes.csv"

### select an interval from
## 1 settings wholeData
interval_comment_whole = "WholeData"
start_time_whole = pd.to_datetime('2014-03-24')
end_time_whole = pd.to_datetime('2015-08-12')

## 2 settings Andrea
interval_comment = "Andrea"
start_time = pd.to_datetime('2014-03-31')
end_time = pd.to_datetime('2014-05-01')

## 3 settings Alina
interval_comment = "Alina"
start_time = pd.to_datetime('2014-03-31')
end_time = pd.to_datetime('2014-11-01')

suffix = "_5sec_"

infile_jobs = datadir + "CPUs/" + interval_comment + "/jobs_cleaned.csv" #01_clean_original_data_jobs

## notice that 43 doesn't exists in the db
#nodes=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64']
nodes=['49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64']
# cpus=['02'] # test


In [6]:
jobs_to_nodes_whole_data = pd.read_csv(infile_jobs_to_nodes, index_col=0)
print("jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

jobs_data = pd.read_csv(infile_jobs)
print("jobs_whole_data contains {} records".format(jobs_data.shape[0]))

jobs_to_nodes_whole_data contains 469095 records
jobs_whole_data contains 336787 records


In [7]:
interval_data_jobs = jobs_data[pd.to_datetime(jobs_data['start_time']) >= start_time]
interval_data_jobs = interval_data_jobs[pd.to_datetime(interval_data_jobs['end_time']) <= end_time]
print("interval_data_jobs contains {} records".format(interval_data_jobs.shape[0]))

interval_data_jobs contains 336787 records


#### solve the problem of jobs runned on node 129

In [8]:
### clean the data
## remove jobs runned on the inexistent node 129
#jobs_to_nodes_whole_data = jobs_to_nodes_whole_data[jobs_to_nodes_whole_data['node_id'] != 129] 

In [9]:
## remove jobs with the same id that runned on the same node
#jobs_to_nodes_whole_data = jobs_to_nodes_whole_data.drop_duplicates(subset=['job_id_string', 'node_id'])
#print("after the clean, jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

In [10]:
### merge the info from the two tables
merged_jobs_to_nodes = pd.merge(interval_data_jobs, jobs_to_nodes_whole_data, how='left', on='job_id_string')
print("merged_jobs_to_nodes contains {} records".format(merged_jobs_to_nodes.shape[0]))

merged_jobs_to_nodes contains 381008 records


In [11]:
# drop the jobs out of time range
#interval_data_jobs = merged_jobs_to_nodes[pd.to_datetime(merged_jobs_to_nodes['run_start_time']) >= start_time]
#interval_data_jobs = interval_data_jobs[pd.to_datetime(interval_data_jobs['end_time']) <= end_time]
#interval_data_jobs = interval_data_jobs.sort_values('run_start_time')
#print("interval_data_jobs contains {} records".format(interval_data_jobs.shape[0]))

In [12]:
merged_jobs_to_nodes.head(1)

,job_id,job_id_string,job_name,queue,start_time,run_start_time,end_time,user,node_req,cpu_req,...,dependency,exit_status,pbs_exit_code,gpu_req,mic_req,node_id,ncpus,ngpus,nmics,mem_requested
0,1,498458.node129,rth2e6t0,parallel,2014-03-31 00:30:02,2014-03-31 00:30:02,2014-03-31 01:22:40,alupi000@node129.eurora.cineca.it,1,16,...,NaN,COMPLETED,NaN,NaN,NaN,17,16,0,0,14680064


In [13]:
node = "33"
print(node)
interval_jobs_to_node = merged_jobs_to_nodes[merged_jobs_to_nodes['node_id'] == int(node)]
print("interval_jobs_node_data contains {} records".format(interval_jobs_to_node.shape[0]))
# read the power mesuraments file obtained with "keep_nodes_data_by_period_1min_sample.py"
infile_node = datadir + "CPUs/" + interval_comment + "/node" + node + suffix + interval_comment

node_data = pd.read_csv(infile_node + ".csv", index_col=1)
print(node_data.shape[0])

33
interval_jobs_node_data contains 9710 records
3715201


In [14]:
node_data.head

<bound method NDFrame.head of                      Unnamed: 0    pow_tot
timestamp                                 
2014-03-31 00:00:00      120960  48.470363
2014-03-31 00:00:05      120961  48.666712
2014-03-31 00:00:10      120962  48.103143
2014-03-31 00:00:15      120963  48.825427
2014-03-31 00:00:20      120964  49.113086
2014-03-31 00:00:25      120965  49.517445
2014-03-31 00:00:30      120966  49.479958
2014-03-31 00:00:35      120967  53.959151
2014-03-31 00:00:40      120968  48.646717
2014-03-31 00:00:45      120969  48.088002
2014-03-31 00:00:50      120970  48.352716
2014-03-31 00:00:55      120971  48.077022
2014-03-31 00:01:00      120972  47.976708
2014-03-31 00:01:05      120973  47.526813
2014-03-31 00:01:10      120974  48.320169
2014-03-31 00:01:15      120975  47.795408
2014-03-31 00:01:20      120976  47.921526
2014-03-31 00:01:25      120977  47.844918
2014-03-31 00:01:30      120978  47.839340
2014-03-31 00:01:35      120979  47.759985
2014-03-31 00:01:40     

In [15]:
#regular_data_jobs = merged_jobs_to_nodes[merged_jobs_to_nodes.node_id >= 1]
#regular_data_jobs = regular_data_jobs[regular_data_jobs.node_id <= 64]
#regular_data_jobs.shape

In [ ]:
for node in nodes:
    print(node)
    interval_jobs_to_node = merged_jobs_to_nodes[merged_jobs_to_nodes['node_id'] == int(node)]
    print("interval_jobs_node_data contains {} records".format(interval_jobs_to_node.shape[0]))
    # read the power mesuraments file obtained with "keep_nodes_data_by_period_1min_sample.py"
    infile_node = datadir + "CPUs/" + interval_comment + "/node" + node + suffix + interval_comment
    
    node_data = pd.read_csv(infile_node + ".csv")
    node_data = node_data.set_index('timestamp')
    print(node_data.shape[0])
    node_start_time = node_data.index[0]
    #node_data.reset_index()
    
    num_minutes = node_data.shape[0]
    active_cores = pd.Series(0, index=np.arange(num_minutes))
    active_jobs = pd.Series(0, index=np.arange(num_minutes))
    active_gpus = pd.Series(0, index=np.arange(num_minutes))
    active_mics = pd.Series(0, index=np.arange(num_minutes))
    
    i = 0
    num_jobs = interval_jobs_to_node.shape[0]
    for index, row in interval_jobs_to_node.iterrows():
        #print("{}/{}   {}".format(i, num_jobs, row['job_id_string']))
        i = i + 1
        job_start_time = pd.to_datetime(row['run_start_time'])
        # print(start_time)
        job_end_time = pd.to_datetime(row['end_time'])
        job_cores = row['ncpus']
        job_gpus = row['ngpus']
        job_mics = row['nmics']
        
        #print(job_start_time)
        #print(pd.to_datetime(node_start_time))
        before_minutes = int((job_start_time - pd.to_datetime(node_start_time)) / np.timedelta64(5, 's'))
        running_minutes = int((job_end_time - job_start_time) / np.timedelta64(5, 's'))
        after_minutes = num_minutes - running_minutes - before_minutes
        #print("{} - {} - {}".format(before_minutes, running_minutes, after_minutes))
        before_serie = pd.Series(0, index=np.arange(before_minutes))
        after_serie = pd.Series(0, index=np.arange(after_minutes))
        running_serie = pd.Series(job_cores, index=np.arange(running_minutes))
        concat_series = pd.concat([before_serie, running_serie, after_serie], ignore_index=True)
        active_cores = active_cores.add(concat_series, fill_value=0)

        running_serie = pd.Series(1, index=np.arange(running_minutes))
        concat_series = pd.concat([before_serie, running_serie, after_serie], ignore_index=True)
        active_jobs= active_jobs.add(concat_series, fill_value=0)

        running_serie = pd.Series(job_gpus, index=np.arange(running_minutes))
        concat_series = pd.concat([before_serie, running_serie, after_serie], ignore_index=True)
        active_gpus = active_gpus.add(concat_series, fill_value=0)

        running_serie = pd.Series(job_mics, index=np.arange(running_minutes))
        concat_series = pd.concat([before_serie, running_serie, after_serie], ignore_index=True)
        active_mics = active_mics.add(concat_series, fill_value=0)
        
    node_data['active_cores'] = active_cores.values
    node_data['active_jobs'] = active_jobs.values
    node_data['active_gpus'] = active_gpus.values
    node_data['active_mics'] = active_mics.values
    
    node_data.to_csv(infile_node + "_active_cores_and_jobs" + ".csv")

49
interval_jobs_node_data contains 12530 records
3715201
50
interval_jobs_node_data contains 4244 records
3715201
